# 3. Reading Data From Kinesis

In [13]:
import boto3
import time
import configparser
import warnings
warnings.filterwarnings("ignore")

In [14]:
config = configparser.ConfigParser()
config.read("config.conf")

glue_db = config.get("athena","glue_db")
schema_name = config.get("athena","schema_name")
stagin_dir = config.get("athena","stagin_dir")
athena_client = boto3.client("athena")

In [ ]:
query_response = athena_client.start_query_execution(
    QueryString=f'SELECT * FROM "{glue_db}"."{schema_name}" limit 10;',
    QueryExecutionContext={"Database": schema_name},
    ResultConfiguration={
        "OutputLocation": stagin_dir,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

In [ ]:
athena_client.get_query_execution(QueryExecutionId=query_response["QueryExecutionId"])

In [ ]:
athena_client.get_query_results(
   QueryExecutionId=query_response["QueryExecutionId"]
)

```python
while True:
    try:
        # This function only loads the first 1000 rows
        athena_client.get_query_results(
            QueryExecutionId=query_response["QueryExecutionId"]
        )
        break
    except Exception as err:
        if "not yet finished" in str(err):
            time.sleep(0.001)
        else:
            raise err
```